In [1]:
import torchreid
print(torchreid.__file__)

c:\users\iot28\deep-person-reid\torchreid\__init__.py


In [ ]:
import cv2
import torch
import numpy as np
import time
from deep_sort_realtime.deepsort_tracker import DeepSort

# 가로 FOV 설정 (예: 60°)
HORIZONTAL_FOV = 60.0
HALF_FOV = HORIZONTAL_FOV / 2.0

def compute_angle(cx, frame_width):
    ratio = (cx - frame_width / 2.0) / (frame_width / 2.0)
    return ratio * HALF_FOV  # -HALF_FOV ~ +HALF_FOV

def main():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True).to(device)
    model.eval()

    tracker = DeepSort(max_age=60, embedder='mobilenet')

    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("카메라 열기 실패")
        return

    W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print(f"Camera resolution: {W}×{H}, Horizontal FOV: {HORIZONTAL_FOV}")

    start_time = time.time()
    user_id = None

    current_angle = 0.0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # YOLOv5로 사람 탐지
        results = model(frame)
        dets = results.xyxy[0].cpu().numpy()

        dets_for_sort = []
        for *box, conf, cls in dets:
            if int(cls) == 0 and conf > 0.3:
                x1, y1, x2, y2 = map(int, box)
                dets_for_sort.append(((x1, y1, x2 - x1, y2 - y1), float(conf)))

        tracks = tracker.update_tracks(dets_for_sort, frame=frame)

        # 초기 5초 동안 가장 큰 사람을 사용자로 지정
        if time.time() - start_time < 5:
            max_area = 0
            for track in tracks:
                if not track.is_confirmed():
                    continue
                x, y, w, h = map(int, track.to_tlwh())
                area = w * h
                if area > max_area:
                    max_area = area
                    user_id = track.track_id

        # 사용자만 표시 & 각도 계산
        for track in tracks:
            if not track.is_confirmed() or track.track_id != user_id:
                continue
            x, y, w, h = map(int, track.to_tlwh())
            cx = x + w / 2.0
            current_angle = compute_angle(cx, W)

            # 빨간 사각형
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)

        # 화면 하단 중앙에 각도 표시
        angle_text = f"{current_angle:+.1f}"
        font = cv2.FONT_HERSHEY_SIMPLEX
        scale = 1.0
        thickness = 2
        (text_w, text_h), _ = cv2.getTextSize(angle_text, font, scale, thickness)
        pos_x = (W - text_w) // 2
        pos_y = H - 20  # 하단에서 20px 위
        cv2.putText(frame, angle_text, (pos_x, pos_y), font, scale, (0, 0, 255), thickness)

        # 중앙선 그리기 (선택)
        cv2.line(frame, (W // 2, 0), (W // 2, H), (255, 0, 0), 1)

        cv2.imshow("User Angle Tracking", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()


Using cache found in C:\Users\IOT28/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-5-30 Python-3.11.11 torch-2.7.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Camera resolution: 640×480, Horizontal FOV: 60.0°


C:\Users\IOT28/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\IOT28/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\IOT28/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\IOT28/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\IOT28/.cache\torch\hub\ultralytics_yolov5_master\models